## 4. Get iso2 Country Codes of Tweets' Location and Sentiment Analysis per Country and Date

### Import packages

In [ ]:
import pandas as pd
import glob
import re
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.reset_option('max_rows')
# pd.reset_option('max_columns')

### Convert location to iso2 country codes

In [ ]:
# geographical data
world_cities = pd.read_csv('../data/worldcities/worldcities.csv', usecols= ['city_ascii', 'country', 'iso2', 'population'])
city_iso2 = world_cities.sort_values('population', ascending = False).drop_duplicates('city_ascii').rename(columns={'city_ascii':'city'})[['city', 'iso2']]
state_iso2 = pd.read_csv('../data/US_states.csv', usecols=['STATE', 'STATE2']).assign(iso2='USA').rename(columns = {'STATE':'state', 'STATE2':'state2'})
country_iso2 = world_cities.sort_values('population', ascending = False).drop_duplicates('iso2')[['country', 'iso2']]

In [ ]:
# Function to replace the inconsistent locations with country names
US = ['US', 'USA', 'Southern California', 'United States of America', 'West Coast, USA (SEA/GEG/LAX)',
       'Democrat in the USA','Washington DC', 'New York City', 'NYC', 'USA (SEA/GEG/LAX)', 'Washington DC',
       'Lost in the Minnesota North Woods', 'Washington D.C.', 'U.S.A.', 'U.S.A', 'us', 'New England', 'Philly','D.C.'
       ,'usa', 'Northern Virginia', 'American', 'South Florida','Pacific Northwest', 'Northern California', 
       'San Diego & New Orleans', 'San Francisco Bay Area','New York and the World', 'America','Washington State', 
       'Midwest','East Coast', 'US of A', 'Harrisburg Pa']
UK = ['UK','England', 'Scotland', 'london', 'Wales', 'uk', 'West Saxons', 'LONDON', 'South London', 'Northern Ireland',
       'Somerset England UK Eu', 'Derbyshire', 'England and International', 'London UK','Leicestershire','Lancashire'
       ,'West Sussex','North Yorkshire','Yorkshire and The Humber, Engl','Yorkshire and The Humber','Newcastle upon Tyne'
       ,'Staffordshire','Oxfordshire','Cardiff','West Yorkshire','Cymru','Great Britain','North Wales','Wiltshire'
       ,'Stockport','North West England','Cambridgeshire','glasgow','Romford','SCOTLAND','South Wales','Dorset'
       ,'some were in UK','england','Hertfordshire','Shropshire England','North East England','Hampshire UK'
       ,'Norfolk England','Dorset, England.','Bristol UK','Republic of Wales','U.K.','Lancashire','East Sussex'
       ,'Warwickshire', 'Edinburgh ~ Heart of Scotland!']
Germany = ["Deutschland", 'Somewhere in Germany']
Ukraine = ['Ucraina', 'chernivtsi', 'Some Future Place in Ukraine', 'Mariouple']
Poland = ['Polska']
Canada = ['CANADA']
Netherlands = ['The Netherlands']
Czechia = ['Czech Republic']
Italy = ['Italia']    
Brazil = ['Brasil', 'Brazil Brazil']   
Finland = ['Suomi', "East-Finnish People's Republic"]    
Japan = ['Okinotorishima Ogasawara Tokyo']   
Australia = ['Queensland','Australia - International']   
Sweden = ['Sverige']  
Kazakhstan = ['Astana']  
Serbia = ['Belgrade City'] 
India = ['mumbai']   
Belgium = ['Belgique', 'Bruxelles']  
Ethiopia = ['Tigray']   
Venezuela = ['Venezuela revolucionaria']
Spain = ['Tarragona']
Greece = ['Athens Greece']
Denmark = ['Danmark']

def convert_loc(text):
    if text in US:
        return 'United States'
    if text in UK:
        return 'United Kingdom'
    if text in Germany:
        return 'Germany'
    if text in Ukraine:
        return 'Ukraine'
    if text in Poland:
        return 'Poland'
    if text in Canada:
        return 'Canada'
    if text in Netherlands:
        return 'Netherlands'
    if text in Czechia:
        return 'Czechia'
    if text in Italy:
        return 'Italy'
    if text in Brazil:
        return 'Brazil'
    if text in Finland:
        return 'Finland'
    if text in Japan:
        return 'Japan'
    if text in Australia:
        return 'Australia'
    if text in Sweden:
        return 'Sweden'
    if text in Kazakhstan:
        return 'Kazakhstan'
    if text in Serbia:
        return 'Serbia'
    if text in India:
        return 'India'
    if text in Belgium:
        return 'Belgium'
    if text in Ethiopia:
        return 'Ethiopia'
    if text in Venezuela:
        return 'Venezuela'
    if text in Spain:
        return 'Spain'
    if text in Greece:
        return 'Greece'
    if text in Denmark:
        return 'Denmark'
    else:
        return text 

In [ ]:
# convert locations to iso2 country codes
col_to_keep = ['tweet_id', 'acct_desc', 'date', 'location', 'friends_ount','followers_ount', 'text', 'compound', 'iso2_final']
for filepath in tqdm(glob.glob('../data/tweets_en/*')):
    df = pd.read_csv(filepath, lineterminator='\n', encoding='latin-1')
    
    # split the location by comma and assign them to loc1 and loc2 
    loc_split = df['location'].apply(lambda x : str(x).split(','))
    loc1 = []
    loc2 = []
    for loc in loc_split:
        if len(loc) == 1:
            loc1 += [loc[0]]
            loc2 += ['nan']
        if len(loc) == 2:
            loc1 += [loc[0]]
            loc2 += [loc[1]]
        if len(loc) >2:
            loc1 += [loc[0]]
            loc2 += [loc[1]]
    df = df.assign(loc1 = loc1, loc2 = loc2)
    
    # strip the blank space in loc1 and loc2; convert the inconsistent loc1 and loc2 to country names
    df['loc1'] = df['loc1'].apply(lambda x: x.strip(' '))
    df['loc1'] = df['loc1'].apply(convert_loc)
    df['loc2'] = df['loc2'].apply(lambda x: x.strip(' '))
    df['loc2'] = df['loc2'].apply(convert_loc)
    
    # merge location columns with iso2 dataframes to match city, state, or country
    df = (df.
     merge(city_iso2, how = 'left', left_on = 'loc1', right_on = 'city').
     merge(city_iso2, how = 'left', left_on = 'loc2', right_on = 'city').
     merge(state_iso2, how = 'left', left_on = 'loc1', right_on = 'state').
     merge(state_iso2, how = 'left', left_on = 'loc2', right_on = 'state').
     merge(state_iso2, how = 'left', left_on = 'loc1', right_on = 'state2').
     merge(state_iso2, how = 'left', left_on = 'loc2', right_on = 'state2').
     merge(country_iso2, how = 'left', left_on = 'loc1', right_on = 'country').
     merge(country_iso2, how = 'left', left_on = 'loc2', right_on = 'country')
    )
    
    # ignore the null values and get the iso2 from iso2_x or iso2_y columns
    df['iso2_final'] = (df['iso2_x'].iloc[:,0].
     combine_first(df['iso2_x'].iloc[:,1]).
     combine_first(df['iso2_x'].iloc[:,2]).
     combine_first(df['iso2_x'].iloc[:,3]).
     combine_first(df['iso2_y'].iloc[:,0]).
     combine_first(df['iso2_y'].iloc[:,1]).
     combine_first(df['iso2_y'].iloc[:,2]).
     combine_first(df['iso2_y'].iloc[:,3])
    )
    df = df[col_to_keep]# drop iso2_ columns
    df.to_csv(filepath, index = False)

### Calculate sentiment scores and number of tweets per country before and after the war (2/24/22)

In [ ]:
startdate =pd.to_datetime('2022-02-24').date()
keywords = ['ukraine', 'russia', 'eu', 'zelenskyy', 'biden', 'putin', 'johnson', 'nato', 'scholz', 'macron']
before_war = pd.DataFrame({'iso2_final':[]}).astype({'iso2_final':'str'})
after_war = pd.DataFrame({'iso2_final':[]}).astype({'iso2_final':'str'})

In [ ]:
for keyword in tqdm(keywords):
    df = pd.read_csv(f'../data/tweets_en/tweets_{keyword}_en.csv',dtype={'date':'str'}, parse_dates = ['date'], lineterminator='\n', encoding='latin-1')
    df['date'] = pd.DatetimeIndex(df['date']).date
    
    # before the war
    df_temp = (df[(df['date']< startdate)&(abs(df['compound'])>0.1)].
                  groupby('iso2_final').
                  mean()[['compound']].
                  reset_index().
                  rename(columns={'compound':f'compound_{keyword}_before_war'}))
    df_temp[f'compound_{keyword}_before_war_count'] = (df[(df['date']< startdate)&(abs(df['compound'])>0.1)].
            groupby('iso2_final').
            count()['compound']).to_list()
    before_war = before_war.merge(df_temp, how='outer', left_on='iso2_final', right_on='iso2_final')
    
    # after the war
    df_temp = (df[(df['date']> startdate)&(abs(df['compound'])>0.1)].
                  groupby('iso2_final').
                  mean()[['compound']].
                  reset_index().
                  rename(columns={'compound':f'compound_{keyword}_after_war'}))
    df_temp[f'compound_{keyword}_after_war_count'] = (df[(df['date']> startdate)&(abs(df['compound'])>0.1)].
            groupby('iso2_final').
            count()['compound']).to_list()
    after_war = after_war.merge(df_temp, how='outer', left_on='iso2_final', right_on='iso2_final')

# merge before_war and after_war, drop columns with all NA, and write to csv    
df_temp = before_war.merge(after_war, how='outer', left_on='iso2_final', right_on='iso2_final')
df_temp.dropna(axis='columns', how='all').to_csv('../shinyapp/data/sentiment_per_country.csv')    

### Calculate sentiment scores and number of tweets per date

In [ ]:
sent_per_date = pd.DataFrame()
for keyword in tqdm(keywords):
    df = pd.read_csv(f'../data/tweets_en/tweets_{keyword}_en.csv',dtype={'date':'str'}, parse_dates = ['date'], lineterminator='\n', encoding='latin-1')
    df['date'] = pd.DatetimeIndex(df['date']).date
    
    # mean sentiment score per date
    df_temp = (df[abs(df['compound'])>0.1].
                  groupby('date').
                  mean()[['compound']].
                  reset_index()
              )
    # number of tweets per date
    df_temp['compound_count'] = (df[abs(df['compound'])>0.1].
                             groupby(['date']).
                             count()['compound'].to_list())
    # add keyword column
    df_temp['keyword'] = keyword
    # concatenate df of different keywords
    sent_per_date = pd.concat([sent_per_date, df_temp], ignore_index=True)

# write to csv
sent_per_date.to_csv('../shinyapp/data/sentiment_per_date.csv') 